In [1]:
import pandas as pd
from symspellpy import SymSpell, Verbosity
import pkg_resources
from Levenshtein import jaro_winkler
import re

count_threshold = 1000

dictionary_path = pkg_resources.resource_filename('symspellpy', 'frequency_dictionary_en_82_765.txt')

en_dict = pd.read_csv(dictionary_path, sep=' ', header=None)

en_dict.columns = ['word', 'freq']

sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7, count_threshold=count_threshold)
dictionary_path = 'review_english_dict.txt'
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1, separator=',', encoding='utf-8')
del dictionary_path

def detect_repeating_characters(text):

    pattern = r'([a-zA-Z])\1\1+'
    regex = re.compile(pattern, re.IGNORECASE)

    return True if len(regex.findall(text)) > 0 else False

print(detect_repeating_characters('aap'))

def enchant(word):
    corrected_word = word

    suggestions = sym_spell.lookup(word, Verbosity.TOP, max_edit_distance=2, include_unknown=False)
    if suggestions:
        terms = [x.term for x in suggestions]
        pairs = []
        for term in terms:
            pairs.append((term, jaro_winkler(word, term, prefix_weight=0.25)))
            sorted(pairs, key=lambda x: (x[1]), reverse=True)

        return corrected_word.replace(word, pairs[0][0])
    else:
        return word
    
print(enchant('glich'))

df = pd.read_csv('review_english_dict_v2.csv')

df = df.dropna()

# remove frequency of 1

df.drop(df[df['freq'] <= 3].index, inplace=True)

# look for words at the bottom and correct them with words at the top

word_pair = {}

for mistake in list(df[df['freq'] < count_threshold]['word']):
    correct = enchant(mistake)
    repeat = detect_repeating_characters(mistake)
    if repeat == True:
        word_pair[mistake] = None
    elif mistake == 'unsubscribtion':
        word_pair[mistake] = 'unsubscription'
    elif mistake == 'aap':
        print('Hello')
        word_pair[mistake] = 'app'
    elif correct != None:
        word_pair[mistake] = correct

df['word'] = df['word'].apply(lambda x: word_pair[x] if x in word_pair.keys() else x)
df = df.groupby('word').sum().sort_values('freq', ascending=False).reset_index()

word_pair['aap'] = 'app'

df['word'] = df['word'].apply(lambda x: word_pair[x] if x in word_pair.keys() else x)
df = df.groupby('word').sum().sort_values('freq', ascending=False).reset_index()

df_complete = pd.concat([en_dict, df])

print(df_complete)
df_complete = df_complete.groupby('word').sum().sort_values('freq', ascending=False).reset_index()
print(df_complete)

df_complete.to_csv('clean_dict_v3.txt', ',', encoding='utf-8', index=False)

/home/denis/miniconda3/envs/cuml_test5/lib/python3.8/site-packages/symspellpy/frequency_dictionary_en_82_765.txt
False
glitch
               word         freq
0               the  23135851162
1                of  13151942776
2               and  12997637966
3                to  12136980858
4                 a   9081174698
...             ...          ...
13805      seaworld            4
13806     avalanche            4
13807    furnitures            4
13808       furnace            4
13809  contributers            4

[96644 rows x 2 columns]
               word         freq
0               the  23136867449
1                of  13152273527
2               and  12998482808
3                to  12137968715
4                 a   9081703301
...             ...          ...
85656     jharkhand            4
85657     rightaway            4
85658        monika            4
85659  monitazation            4
85660        dunkin            4

[85661 rows x 2 columns]
